In [1]:
import os
import sys
import re
import time
import random
import warnings
import collections
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

sys.path.append('../../src')
import cb_utils

sns.set(style="darkgrid")
pd.options.display.max_columns = 500

%load_ext autoreload
%autoreload 2

# Milliman PAC File ETL
1) log into SFTP site: https://secureftp.milliman.com  
2) In the "from milliman" folder, go to the latest directory  
     - In this case, the latest directory is "20221216"  
3) Make sure you have access to the following folder: \\Main Street Health\MSH_Strategy_Group - ACO Analyses\Raw MIlliman Flat FIles\  
     - create folder with same datestamp as file from milliman, in this case "20221216"  
4) Download the files from the SFTP site  
     - in this case there are 3  
5) copy the data disctionary to the folder on the network drive.  
6) download the prospective and retrospective CSV files into database tables in both the MSH and CB datagrip environments (Alan has a place he puts them)  
7) delete the last 30 columns from these database tables (we will create a "pivot file" with these 30 columns):  
8) with these 30 columns, run Alan's "pivot code" to create these files:  
     - strategic_milliman_prosp_20221128_pivot
     -  strategic_milliman_retro_20221128_pivot
9) copy 4 files to the "20221216" folder  
     - prospective flat file (without the 30 MA columns)
     - retrospective flat file (without the 30 MA columns)
     - prospective MA pivot file
     - retrospective MA pivor file
10) email Ali and Ben that process is complete with the following summaries that you can pull via queries and paste in excel into the same folder.  

### Config

In [2]:
data_dir = '/Users/bp/workspace/msh/milliman_data'
current_folder = '20250401'
pro_file_name = 'ACO Builder MSSP Prospective Explorer v2026.2.txt'
retro_file_name = 'ACO Builder MSSP Retrospective Explorer v2026.2.txt'

### Pro load

In [3]:
pro_df = pd.read_csv(f'{data_dir}/{current_folder}/{pro_file_name}',  encoding='latin1', low_memory=False, delimiter='\t')
pro_df = cb_utils.df_format_columns(pro_df)
print(f'{pro_df.shape[0]} rows, {pro_df.shape[1]} columns')
pro_df.head()

764493 rows, 215 columns


,pac_id,deid_tin,tin_public,year,program_type,provider_name,mssp_aco_name,reach_aco_name,practice_state,practice_msa,practice_st_msa,practice_msa_name,practice_zip,spec_count,np_count,cns_count,pcp_count,pa_count,other_count,unique_members,pys_esrd,pys_dis,pys_ad,pys_and,pys_total,rs_esrd_legacy,rs_dis_legacy,rs_ad_legacy,rs_and_legacy,risk_composite_legacy,rs_esrd_v24,rs_dis_v28,rs_ad_v28,rs_and_v28,risk_composite_v28,rs_esrd_demog,rs_dis_demog,rs_ad_demog,rs_and_demog,risk_composite_demog,paid_esrd,paid_dis,paid_ad,paid_and,paid_composite,reg_ret_exp_esrd,reg_ret_exp_dis,reg_ret_exp_ad,reg_ret_exp_and,reg_ret_exp_composite,regional_efficiency_ret,reg_pro_exp_esrd,reg_pro_exp_dis,reg_pro_exp_ad,reg_pro_exp_and,reg_pro_exp_composite,regional_efficiency_pro,reg_ret_exp_esrd_v24,reg_ret_exp_dis_v28,reg_ret_exp_ad_v28,reg_ret_exp_and_v28,reg_ret_exp_composite_v28,regional_efficiency_ret_v28,reg_pro_exp_esrd_v24,reg_pro_exp_dis_v28,reg_pro_exp_ad_v28,reg_pro_exp_and_v28,reg_pro_exp_composite_v28,regional_efficiency_pro_v28,ancillary_paid,hospice_paid,homehealthcare_paid,inpatient_paid,miscellaneousmedical_paid,officeadministereddr_paid,officehomeurgentvisi_paid,otherpreventive_paid,othervisits_paid,inpatientvisits_paid,professionaldialysis_paid,psychiatricsud_paid,radiologypathologyla_paid,surgery_paid,professionaltotal_paid,emergencydepartment_paid,outpatientdialysis_paid,outpatientpharmacy_paid,outpatientpsychiatri_paid,outpatientother_paid,outpatienttotal_paid,skillednursingfacili_paid,miscellaneousmedical_paid_keep,officeadministereddr_paid_keep,officehomeurgentvisi_paid_keep,otherpreventive_paid_keep,othervisits_paid_keep,inpatientvisits_paid_keep,professionaldialysis_paid_keep,psychiatricsud_paid_keep,radiologypathologyla_paid_keep,surgery_paid_keep,professionaltotal_paid_keep,outpatienttotal_paid_keep,ancillary_visits,hospice_visits,homehealthcare_visits,inpatient_admit,inpatient_day,miscellaneousmedical_proc,officeadministereddr_proc,otherpreventive_proc,professionaldialysis_proc,radiologypathologyla_proc,surgery_proc,emergencydepartment_visits,outpatientdialysis_visits,outpatientpharmacy_visits,outpatientpsychiatri_visits,outpatient_visits,skillednursingfacili_admit,skillednursingfacili_day,outpatient_visits_keep,miscellaneousmedical_proc_keep,officeadministereddr_proc_keep,otherpreventive_proc_keep,professionaldialysis_proc_keep,radiologypathologyla_proc_keep,surgery_proc_keep,alcohol,drug,edcnnpa,edcnpa,epct,injury,noner,psych,unclassified,readmissions,awv,hra,tel,awv_keep,hra_keep,tel_keep,pqem_costs,pqem_costs_keep,pys_esrd_pcp,pys_dis_pcp,pys_ad_pcp,pys_and_pcp,mssp_paid_esrd_pcp,mssp_paid_dis_pcp,mssp_paid_ad_pcp,mssp_paid_and_pcp,rs_esrd_pcp,rs_dis_pcp,rs_ad_pcp,rs_and_pcp,pys_esrd_spc,pys_dis_spc,pys_ad_spc,pys_and_spc,mssp_paid_esrd_spc,mssp_paid_dis_spc,mssp_paid_ad_spc,mssp_paid_and_spc,rs_esrd_spc,rs_dis_spc,rs_ad_spc,rs_and_spc,pys_esrd_st3,pys_dis_st3,pys_ad_st3,pys_and_st3,mssp_paid_esrd_st3,mssp_paid_dis_st3,mssp_paid_ad_st3,mssp_paid_and_st3,rs_esrd_st3,rs_dis_st3,rs_ad_st3,rs_and_st3,unique_members_potltp,pys_esrd_potltp,pys_dis_potltp,pys_ad_potltp,pys_and_potltp,mssp_paid_esrd_potltp,mssp_paid_dis_potltp,mssp_paid_ad_potltp,mssp_paid_and_potltp,rs_esrd_potltp,rs_dis_potltp,rs_ad_potltp,rs_and_potltp,pys_esrd_nomd,pys_dis_nomd,pys_ad_nomd,pys_and_nomd,mssp_paid_esrd_nomd,mssp_paid_dis_nomd,mssp_paid_ad_nomd,mssp_paid_and_nomd,rs_esrd_nomd,rs_dis_nomd,rs_ad_nomd,rs_and_nomd,pys_office,pys_outpatient,pys_snf,pys_nf,pys_hcbs,pys_alf,pys_fqhc_rhc,pys_other
0,Not Available,0000B368EDD2,201627675,2016,Prospective,PALO VERDE CLINIC PLLC,NaN,NaN,Arizona,38060.0,AZ-38060,"Phoenix-Mesa-Chandler, AZ",85224.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0,0.00,1.00,1.00,NaN,NaN,NaN,0.278389,0.278389,NaN,NaN,NaN,0.278389,0.278389,NaN,NaN,NaN,0.608052,0.608052,NaN,NaN,NaN,285.940000,285.940000,0.00,0.00,0.00,9819.80,9819.800000,0.104597,0.00,0.00,0.00,9560.63,9560.630000,0.107432,0.00,0.00,0.00,9819.80,9819.800000,0.104597,0.0

In [4]:
conn = cb_utils.get_engine(source='msh_analytics')
pro_table_name = f'milliman_pro_{current_folder}'
pro_schema = 'raw'
print(f'select * from {pro_schema}.{pro_table_name}')
# pro_df.to_sql(pro_table_name, conn, schema=pro_schema, index=False, method='multi', chunksize=1000) # , if_exists='replace'

select * from raw.milliman_pro_20250401


In [6]:
pro_df.to_sql(pro_table_name, conn, schema=pro_schema, index=False, method='multi', chunksize=1000) # , if_exists='replace'

764493

In [7]:
ret_df = pd.read_csv(f'{data_dir}/{current_folder}/{retro_file_name}',  encoding='latin1', low_memory=False, delimiter='\t')
ret_df = cb_utils.df_format_columns(ret_df)
print(f'{ret_df.shape[0]} rows, {ret_df.shape[1]} columns')
ret_df.head()

739405 rows, 215 columns


,pac_id,deid_tin,tin_public,year,program_type,provider_name,mssp_aco_name,reach_aco_name,practice_state,practice_msa,practice_st_msa,practice_msa_name,practice_zip,spec_count,np_count,cns_count,pcp_count,pa_count,other_count,unique_members,pys_esrd,pys_dis,pys_ad,pys_and,pys_total,rs_esrd_legacy,rs_dis_legacy,rs_ad_legacy,rs_and_legacy,risk_composite_legacy,rs_esrd_v24,rs_dis_v28,rs_ad_v28,rs_and_v28,risk_composite_v28,rs_esrd_demog,rs_dis_demog,rs_ad_demog,rs_and_demog,risk_composite_demog,paid_esrd,paid_dis,paid_ad,paid_and,paid_composite,reg_ret_exp_esrd,reg_ret_exp_dis,reg_ret_exp_ad,reg_ret_exp_and,reg_ret_exp_composite,regional_efficiency_ret,reg_pro_exp_esrd,reg_pro_exp_dis,reg_pro_exp_ad,reg_pro_exp_and,reg_pro_exp_composite,regional_efficiency_pro,reg_ret_exp_esrd_v24,reg_ret_exp_dis_v28,reg_ret_exp_ad_v28,reg_ret_exp_and_v28,reg_ret_exp_composite_v28,regional_efficiency_ret_v28,reg_pro_exp_esrd_v24,reg_pro_exp_dis_v28,reg_pro_exp_ad_v28,reg_pro_exp_and_v28,reg_pro_exp_composite_v28,regional_efficiency_pro_v28,ancillary_paid,hospice_paid,homehealthcare_paid,inpatient_paid,miscellaneousmedical_paid,officeadministereddr_paid,officehomeurgentvisi_paid,otherpreventive_paid,othervisits_paid,inpatientvisits_paid,professionaldialysis_paid,psychiatricsud_paid,radiologypathologyla_paid,surgery_paid,professionaltotal_paid,emergencydepartment_paid,outpatientdialysis_paid,outpatientpharmacy_paid,outpatientpsychiatri_paid,outpatientother_paid,outpatienttotal_paid,skillednursingfacili_paid,miscellaneousmedical_paid_keep,officeadministereddr_paid_keep,officehomeurgentvisi_paid_keep,otherpreventive_paid_keep,othervisits_paid_keep,inpatientvisits_paid_keep,professionaldialysis_paid_keep,psychiatricsud_paid_keep,radiologypathologyla_paid_keep,surgery_paid_keep,professionaltotal_paid_keep,outpatienttotal_paid_keep,ancillary_visits,hospice_visits,homehealthcare_visits,inpatient_admit,inpatient_day,miscellaneousmedical_proc,officeadministereddr_proc,otherpreventive_proc,professionaldialysis_proc,radiologypathologyla_proc,surgery_proc,emergencydepartment_visits,outpatientdialysis_visits,outpatientpharmacy_visits,outpatientpsychiatri_visits,outpatient_visits,skillednursingfacili_admit,skillednursingfacili_day,outpatient_visits_keep,miscellaneousmedical_proc_keep,officeadministereddr_proc_keep,otherpreventive_proc_keep,professionaldialysis_proc_keep,radiologypathologyla_proc_keep,surgery_proc_keep,alcohol,drug,edcnnpa,edcnpa,epct,injury,noner,psych,unclassified,readmissions,awv,hra,tel,awv_keep,hra_keep,tel_keep,pqem_costs,pqem_costs_keep,pys_esrd_pcp,pys_dis_pcp,pys_ad_pcp,pys_and_pcp,mssp_paid_esrd_pcp,mssp_paid_dis_pcp,mssp_paid_ad_pcp,mssp_paid_and_pcp,rs_esrd_pcp,rs_dis_pcp,rs_ad_pcp,rs_and_pcp,pys_esrd_spc,pys_dis_spc,pys_ad_spc,pys_and_spc,mssp_paid_esrd_spc,mssp_paid_dis_spc,mssp_paid_ad_spc,mssp_paid_and_spc,rs_esrd_spc,rs_dis_spc,rs_ad_spc,rs_and_spc,pys_esrd_st3,pys_dis_st3,pys_ad_st3,pys_and_st3,mssp_paid_esrd_st3,mssp_paid_dis_st3,mssp_paid_ad_st3,mssp_paid_and_st3,rs_esrd_st3,rs_dis_st3,rs_ad_st3,rs_and_st3,unique_members_potltp,pys_esrd_potltp,pys_dis_potltp,pys_ad_potltp,pys_and_potltp,mssp_paid_esrd_potltp,mssp_paid_dis_potltp,mssp_paid_ad_potltp,mssp_paid_and_potltp,rs_esrd_potltp,rs_dis_potltp,rs_ad_potltp,rs_and_potltp,pys_esrd_nomd,pys_dis_nomd,pys_ad_nomd,pys_and_nomd,mssp_paid_esrd_nomd,mssp_paid_dis_nomd,mssp_paid_ad_nomd,mssp_paid_and_nomd,rs_esrd_nomd,rs_dis_nomd,rs_ad_nomd,rs_and_nomd,pys_office,pys_outpatient,pys_snf,pys_nf,pys_hcbs,pys_alf,pys_fqhc_rhc,pys_other
0,9335446202,00004B7C4023,474454561,2016,Retrospective,RINDFLEISCH FAMILY PRACTICE PLLC,NaN,"PEARL PRIMARY CARE NETWORK, LLC",Idaho,26820.0,ID-26820,"Idaho Falls, ID",83404.0,NaN,1.0,NaN,1.0,1.0,0.0,151.0,2.00,16.42,13.67,109.08,141.17,1.089615,0.970784,0.861216,1.083805,1.049647,1.089615,0.970784,0.861216,1.083805,1.049647,1.190661,0.961259,1.030802,1.155459,1.124938,56585.550000,17678.937272,23318.405999,21050.820315,21381.634483,69821.11,10583.22,16527

In [8]:
ret_table_name = f'milliman_ret_{current_folder}'
ret_schema = 'raw'
print(f'select * from {ret_schema}.{ret_table_name}')

select * from raw.milliman_ret_20250401


In [9]:
ret_df.to_sql(f'milliman_ret_{current_folder}', conn, schema='raw', index=False, method='multi', chunksize=1000) # if_exists='replace', 

739405

In [ ]:
ret_df.year.value_counts()

In [10]:
for yr in [2016, 2017, 2018, 2019, 2020, 2021, 2022]:
    ret_df.loc[ret_df.year == yr].to_csv(f'{data_dir}/{current_folder}/ret_{yr}.csv',index=False)

# Did not use below

In [ ]:
conn = cb_utils.get_engine()
pro_df.to_sql(f'milliman_pro_{current_folder}', conn, schema='strategic', index=False, method='multi', chunksize=1000) # , if_exists='replace'

### Retro load

In [33]:

ret_df.to_csv('ret.csv',index=False)

In [ ]:
conn = cb_utils.get_engine()
ret_df.to_sql(f'milliman_ret_{current_folder}', conn, schema='strategic', index=False, method='multi', chunksize=1000) # if_exists='replace', 

### Pivots

In [11]:
def pivot_ma_plan_enrollment(df):
    dfs = []
    for i in range(1, 16):
        name, enrollment = f'ma_plan_{i}_name', f'ma_plan_{i}_enrollment'
        plan = df[['deid_tin', 'pac_id', 'year', name, enrollment]]
        plan = plan.loc[~(plan[name].isna()) & ~(plan[enrollment].isna())]
        plan.columns = ['deid_tin', 'pac_id', 'year', 'ma_plan_name', 'ma_enrollment']
        dfs.append(plan)
    return pd.concat(dfs)

In [ ]:
ret_pivot_df = pivot_ma_plan_enrollment(ret_df)

In [ ]:
ret_pivot_df.head()

In [ ]:
pro_pivot_df = pivot_ma_plan_enrollment(pro_df)

In [ ]:
pro_pivot_df.head()

### Summaries 
### Prospective

In [ ]:
s1 = ret_df[['year', 'unique_members']].groupby('year', as_index=False).sum()
s1.head()

### Retro

In [ ]:
s2 = pro_df[['year', 'unique_members']].groupby('year', as_index=False).sum()
s2.head()

### Prospective

In [ ]:
s3 = pro_pivot_df[['year', 'ma_plan_name', 'ma_enrollment']].groupby(['year', 'ma_plan_name'], as_index=False).sum().sort_values('ma_enrollment', ascending=False)
s3.head()

### Retro

In [ ]:
s4 = ret_pivot_df[['year', 'ma_plan_name', 'ma_enrollment']].groupby(['year', 'ma_plan_name'], as_index=False).sum().sort_values('ma_enrollment', ascending=False)
s4.head()

### Outputs

In [ ]:
ret_df.loc[:, ret_df.columns[:-30]].to_csv(f'{data_dir}/{current_folder}/ours_milliman_ret_{current_folder}.csv', index=False)

In [ ]:
pro_df.loc[:, pro_df.columns[:-30]].to_csv(f'{data_dir}/{current_folder}/ours_milliman_pro_{current_folder}.csv', index=False)

In [ ]:
pro_pivot_df.to_csv(f'{data_dir}/{current_folder}/pro_pivot.csv', index=False)
ret_pivot_df.to_csv(f'{data_dir}/{current_folder}/ret_pivot.csv', index=False)

In [ ]:
with pd.ExcelWriter(f'{data_dir}/{current_folder}/summary.xlsx') as writer:
    s1.to_excel(writer, sheet_name='retrospective', index=False)
    s2.to_excel(writer, sheet_name='prospective', index=False)
    s3.to_excel(writer, sheet_name='prospective_pivot', index=False)
    s4.to_excel(writer, sheet_name='retrospective_pivot', index=False)